In [1]:
from pathlib import Path
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool
from langchain.embeddings import OpenAIEmbeddings
from langchain_experimental.autonomous_agents import AutoGPT

from langchain_setup import ChatOpenAI, tracing_v2_enabled_if_api_key_set
from langchain_setup.qdrant import create_inmemory_empty_qdrant

# Tools

In [2]:
root_dir = str(Path('../data/autogpt/').resolve())

search = DuckDuckGoSearchAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(root_dir=root_dir),
    ReadFileTool(root_dir=root_dir),
]

# Memory
儲存 agent 在執行過程中產生的文檔、紀錄，方便之後抽取 (retrieval)

In [3]:
vectorstore = create_inmemory_empty_qdrant()

# 建立 AutoGPT

In [4]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
    # chat_history_memory= default using ChatMessageHistory
)

# Run

In [5]:
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    agent.run(["Write a brief investment analysis report for Taiwan Semiconductor Manufacturing Co., Ltd. (TSMC)."])

{
  "thoughts": {
    "text": "Since I don't have any specific tasks at the moment, I should start by gathering information about Taiwan Semiconductor Manufacturing Co., Ltd. (TSMC) for the investment analysis report. I can use the 'search' command to find relevant information.",
    "reasoning": "I need to gather information about TSMC to analyze its investment potential.",
    "plan": "- Use the 'search' command to find information about TSMC\n- Analyze the gathered information\n- Write the investment analysis report",
    "criticism": "I need to make sure I gather accurate and up-to-date information about TSMC to provide an accurate analysis.",
    "speak": "I will start by searching for information about Taiwan Semiconductor Manufacturing Co., Ltd. (TSMC) to gather data for the investment analysis report."
  },
  "command": {
    "name": "search",
    "args": {
      "tool_input": "Taiwan Semiconductor Manufacturing Co., Ltd. (TSMC)"
    }
  }
}
{
  "thoughts": {
    "text": "Now t

# Explain

AutoGPT 有兩個 memory, `memory` (vectorstore of Historical AI respones) 和 `chat_history_memory` (messages of chat history)
1. 在構成 Prompt 時，會以前 10 messages 從 `memory` 抽取相關的過去的 AI 回答，將過去的 AI 回答以相關性由高到低插入 Prompt 的 AI reply 段落中，Prompt 中原本寫好的設定和抽取進來的過去 AI 回答加起來不超過 2500。
2. 之後對於 `chat_history_memory` 中 Human, AI, System 的對話紀錄，會由新至舊一路加到不超過 prompt 的 token 數限制為止。